# Audiobooks business case

### Import the relevant libraries

In [1]:
import numpy as np
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

### Data extraction

In [2]:
# let's create a temporary variable npz, where the three Audiobooks datasets will be stored.
npz = np.load('/kaggle/input/audiobook/Audiobooks_data_train.npz')

# Extract the inputs, and change them to float type.
train_inputs = npz['inputs'].astype(np.float)
# Targets must be int because of the sparse_categorical_crossentropy loss function, so they can be easily one-hot encoded
train_targets = npz['targets'].astype(np.int)

# Load the validation data in the temporary variable and extract intputs and targets
npz = np.load('/kaggle/input/audiobook/Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# Same with the test data
npz = np.load('/kaggle/input/audiobook/Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

/tmp/ipykernel_33/549222847.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
/tmp/ipykernel_33/549222847.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_targets = 

### Model creation

In [30]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers.
hidden_layer_size = 32
    
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'), # 3rd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 64
max_epochs = 30

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size,
          epochs=max_epochs, # Epochs that we will train for (assuming there is not early stopping)
          # Callbacks are functions called when a task is completed
          # In this case, the task is to check if val_loss is increasing
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), #I could also have splitted the data into to npz (train and test) 
          # and then make the split with validation_split
          verbose = 1
          )  

Epoch 1/30
56/56 [==============================] - 1s 6ms/step - loss: 0.6399 - accuracy: 0.6577 - val_loss: 0.5401 - val_accuracy: 0.7651
Epoch 2/30
56/56 [==============================] - 0s 3ms/step - loss: 0.4855 - accuracy: 0.7572 - val_loss: 0.4231 - val_accuracy: 0.7673
Epoch 3/30
56/56 [==============================] - 0s 2ms/step - loss: 0.4109 - accuracy: 0.7779 - val_loss: 0.3838 - val_accuracy: 0.7830
Epoch 4/30
56/56 [==============================] - 0s 2ms/step - loss: 0.3826 - accuracy: 0.7890 - val_loss: 0.3595 - val_accuracy: 0.8233
Epoch 5/30
56/56 [==============================] - 0s 3ms/step - loss: 0.3701 - accuracy: 0.7935 - val_loss: 0.3523 - val_accuracy: 0.7852
Epoch 6/30
56/56 [==============================] - 0s 3ms/step - loss: 0.3590 - accuracy: 0.8044 - val_loss: 0.3387 - val_accuracy: 0.8412
Epoch 7/30
56/56 [==============================] - 0s 2ms/step - loss: 0.3515 - accuracy: 0.8027 - val_loss: 0.3342 - val_accuracy: 0.8098
Epoch 8/30
56/56 [==

## Testing the model

After training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

In [31]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3320 - accuracy: 0.8326


In [32]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.33. Test accuracy: 83.26%
